In [1]:
import os
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

import chromedriver_binary
from selenium import webdriver


link = 'https://www.imdb.com/search/title/?genres=war'

### 1. Using Requests

In [2]:
link = 'https://www.imdb.com/search/title/?genres=war'
res = requests.get(link)
soup = BeautifulSoup(res.content, 'html.parser')

data = []

page = 0

while (soup.find('a',class_ = 'lister-page-next next-page').get('href')):

    votes = np.nan
    gross = np.nan

    for m in soup.find('div', class_ = 'lister-list').find_all('div', class_ = 'lister-item mode-advanced'):

        try:
            id_         = m.find('div', class_ = 'lister-item-image float-left').find('a').get('href').split('/')[-2].strip()
        except:
            id_         = np.nan

        try:
            name        = m.find('h3').find('a').text
        except:
            name        = np.nan

        try:
            year        =  m.find('h3').find('span', class_ = 'lister-item-year').text
        except:
            year        = np.nan

        try:
            certificate = m.find('p', class_ = "text-muted").find('span', class_ = 'certificate').text.strip()
        except:
            certificate = np.nan

        try:
            duration    = m.find('p', class_ = "text-muted").find('span', class_ = 'runtime').text.strip()
        except:
            duration    = np.nan

        try:    
            genre       = m.find('p', class_ = "text-muted").find('span', class_ = 'genre').text.strip()
        except:
            genre       = np.nan

        try:
            rating      = m.find('div', class_ = 'ratings-bar').find('strong').text.strip()
        except:
            rating      = np.nan

        try:
            description = m.find_all('p', class_ = 'text-muted')[1].text.strip()
        except:
            description = np.nan



        try:     # If were having both directors and stars
            temp = BeautifulSoup(str(m.find_all('p')[2]).split('<span class="ghost">|</span>')[0], 'html.parser')
            if ('Director' in temp.text):

                directors_id   = ','.join([i.get('href').split('/')[-2] for i in temp.find_all('a')])
                directors_name = ','.join([i.text.strip() for i in temp.find_all('a')])

            temp = BeautifulSoup(str(m.find_all('p')[2]).split('<span class="ghost">|</span> ')[1], 'html.parser')
            if ("Star" in temp.text):

                stars_id   = ','.join([i.get('href').split('/')[-2] for i in temp.find_all('a')])
                stars_name = ','.join([i.text.strip() for i in temp.find_all('a')])
        except:

            directors_id   = np.nan
            stars_id       = np.nan
            directors_name = np.nan
            stars_name     = np.nan


        try:       # Directors but not Stars
            if('Director' in m.find_all('p')[2].text and 'Stars' not in m.find_all('p')[2].text):
                directors_id   = ','.join([i.get('href') for i in m.find_all('p')[2].find_all('a')])
                directors_name = ','.join([i.text.strip() for i in m.find_all('p')[2].find_all('a')])
        except:
            directors_id = np.nan
            directors_name = np.nan

        try:        # Stars but not Directors
            if('Stars' in m.find_all('p')[2].text and 'Director' not in m.find_all('p')[2].text):
                stars_id     = ','.join([i.get('href').split('/')[-2] for i in m.find_all('p')[2].find_all('a')])
                stars_name   = ','.join([i.text.strip() for i in m.find_all('p')[2].find_all('a')])
        except:
            stars_id = np.nan
            stars_name = np.nan


        try:
            if len(m.find('p', class_ = 'sort-num_votes-visible').find_all('span')) == 2:

                if('Vote' in m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[0].text):    
                    votes =  m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].text.strip()

                elif('Gros' in m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[0].text):    
                    gross =  m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].text.strip()

            else:
                votes = m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[1].get('data-value')
                gross = m.find('p', class_ = 'sort-num_votes-visible').find_all('span')[4].get('data-value')
        except:

            votes = np.nan
            gross = np.nan

        data.append([id_, name, year, rating ,certificate, duration, genre, votes , gross ,directors_id, 
                     directors_name , stars_id, stars_name, description])

        
        
    next_page_link = 'https://www.imdb.com' + soup.find('a',class_ = 'lister-page-next next-page').get('href')

    res = requests.get(next_page_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    page += 1
    
    if (page % 10 == 0):
        print(page ,'pages scraped')
        df = pd.DataFrame(data, columns = ['id', 'name', 'year', 'rating' ,'certificate', 'duration', 'genre', 'votes'
                                   , 'gross_income' ,'directors_id', 'directors_name' , 'stars_id', 'stars_name', 
                                   'description'])
        df.to_csv('checkpoints/' + str(page) + '.csv', index = False)
        
        data = []

10 pages scraped
20 pages scraped
30 pages scraped
40 pages scraped
50 pages scraped
60 pages scraped
70 pages scraped
80 pages scraped
90 pages scraped
100 pages scraped
110 pages scraped
120 pages scraped
130 pages scraped
140 pages scraped
150 pages scraped
160 pages scraped
170 pages scraped
180 pages scraped
190 pages scraped
200 pages scraped
210 pages scraped
220 pages scraped
230 pages scraped
240 pages scraped
250 pages scraped
260 pages scraped
270 pages scraped
280 pages scraped
290 pages scraped
300 pages scraped
310 pages scraped
320 pages scraped
330 pages scraped
340 pages scraped
350 pages scraped
360 pages scraped
370 pages scraped
380 pages scraped
390 pages scraped
400 pages scraped
410 pages scraped
420 pages scraped
430 pages scraped
440 pages scraped
450 pages scraped
460 pages scraped
470 pages scraped
480 pages scraped
490 pages scraped
500 pages scraped
510 pages scraped
520 pages scraped
530 pages scraped
540 pages scraped
550 pages scraped
560 pages scraped
5

KeyboardInterrupt: 

In [3]:
df = pd.DataFrame(data, columns = ['id', 'name', 'year', 'rating' ,'certificate', 'duration', 'genre', 'votes'
                                   , 'gross_income' ,'directors_id', 'directors_name' , 'stars_id', 'stars_name', 
                                   'description'])

df.head()

df.to_csv('checkpoints/'+ str(page) +'.csv', index = False)

### 2. Merging Dataset

In [4]:
df = pd.DataFrame()

files = []

for i in os.listdir('checkpoints/'):                # Taking all files in Sorted Format    
    if(i != '.DS_Store'):
        files.append(int(i.split('.')[0]))
        
files = sorted(files)

for i in tqdm(files):                                    # Taking each file one by one
    file_name = 'checkpoints/' + str(i) + '.csv'
    df_ = pd.read_csv(file_name)                   # Loading the Data
    df = pd.concat((df,df_))                       # Concatinating the Data
    
df.to_csv('war.csv', index = False)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 32.98it/s]
